## 安装环境

In [1]:
!pip install gymnasium
!pip install minigrid
!pip install gym-minigrid

In [ ]:
# from main import MiniGridLLMAgent
# import gymnasium as gym
# import minigrid  # 此行必须存在以注册环境

# # 创建环境
# env = gym.make("MiniGrid-Empty-5x5-v0", render_mode="human")
# agent = MiniGridLLMAgent(env_name="MiniGrid-Empty-5x5-v0")

In [ ]:
# from minigrid import wrappers
# print(dir(wrappers))

In [ ]:
# # 验证可用环境列表
# from gymnasium.envs.registration import registry

# minigrid_envs = [env_id for env_id in registry if env_id.startswith('MiniGrid')]
# print("Available MiniGrid environments:")
# for env_id in sorted(minigrid_envs):
#     print(f"- {env_id}")

In [ ]:
# import gymnasium as gym
# print(gym.pprint_registry())

## 评估

In [2]:
from main import *

In [3]:
evaluation_results = evaluate_llm(
    env_names=[
        "MiniGrid-Empty-5x5-v0",
    ],
    seeds=[42],
    max_steps=10
)


=== Evaluating on MiniGrid-Empty-5x5-v0 ===
Move blocked at (2, 1) facing 3
Seed 42: Steps: 10 | Reward: 0.00

=== Final Evaluation Report ===

Environment: MiniGrid-Empty-5x5-v0
Average Steps: 10.0
Average Reward: 0.00
Invalid Actions per Episode: 0.0


In [4]:
evaluation_results

{'MiniGrid-Empty-5x5-v0': {'avg_steps': 10.0,
  'avg_reward': 0.0,
  'invalid_actions': 0,
  'conversations': [[{'role': 'system',
     'content': "You are an AI agent navigating a grid environment. Your task is:\n1. Analyze environmental observations\n2. Choose optimal actions to achieve the mission\n\n**Core Rules**:\n- Strictly use ONLY the provided action list\n- Prioritize shortest paths\n- Avoid dangerous areas (lava)\n- Manage inventory carefully (keys, etc)\n- Closed doors require 'toggle' to open\n- Locked doors need matching key (use pickup first)\n\n**Action Space** (REQUIRED RESPONSE FORMAT):\nturn_left   : Rotate 90° counter-clockwise\nturn_right  : Rotate 90° clockwise\nmove_forward: Advance if path clear\npickup      : Collect keys/objects\ndrop        : Drop carried object\ntoggle      : Open doors or interact (facing target required)\ndone        : ONLY when goal reached\n\n**Observation**:\nYou receive the entire grid as an observation, represented as a 3D array of sh

## 评估v2

In [5]:
from main import *

In [6]:
evaluation_results = evaluate_llm(
    env_names=[
        "MiniGrid-Empty-5x5-v0",
        "MiniGrid-DoorKey-6x6-v0",
        "MiniGrid-LavaGapS5-v0"
    ],
    seeds=[42, 123, 456, 789],  # 4个随机种子
    max_steps=15 # 对于困难任务可以增加
)


=== Evaluating on MiniGrid-Empty-5x5-v0 ===
Seed 42: Steps: 7 | Reward: 0.58
Seed 123: Steps: 7 | Reward: 0.58
Seed 456: Steps: 7 | Reward: 0.58
Move blocked at (2, 3) facing 1
Move blocked at (2, 3) facing 1
Move blocked at (2, 3) facing 1
Seed 789: Steps: 14 | Reward: 0.16

=== Evaluating on MiniGrid-DoorKey-6x6-v0 ===
Move blocked at (1, 3) facing 0
Move blocked at (1, 2) facing 0
Seed 42: Steps: 15 | Reward: 0.00
Move blocked at (1, 3) facing 0
Move blocked at (1, 4) facing 0
Move blocked at (1, 3) facing 3
Move blocked at (1, 3) facing 0
Seed 123: Steps: 15 | Reward: 0.00
Move blocked at (1, 3) facing 0
Move blocked at (1, 3) facing 0
Move blocked at (1, 3) facing 0
Move blocked at (1, 3) facing 0
Move blocked at (1, 3) facing 0
Move blocked at (1, 3) facing 0
Move blocked at (1, 3) facing 0
Seed 456: Steps: 15 | Reward: 0.00
Move blocked at (2, 2) facing 0
Move blocked at (2, 1) facing 0
Seed 789: Steps: 15 | Reward: 0.00

=== Evaluating on MiniGrid-LavaGapS5-v0 ===
Move blocked

In [11]:
evaluation_results

{'MiniGrid-Empty-5x5-v0': {'avg_steps': 8.75,
  'avg_reward': 0.4749999999999999,
  'invalid_actions': 0,
  'conversations': [[{'role': 'system',
     'content': "You are an AI agent navigating a grid environment. Your task is:\n1. Analyze environmental observations\n2. Choose optimal actions to achieve the mission\n\n**Core Rules**:\n- Strictly use ONLY the provided action list\n- Prioritize shortest paths\n- Avoid dangerous areas (lava)\n- Manage inventory carefully (keys, etc)\n- Closed doors require 'toggle' to open\n- Locked doors need matching key (use pickup first)\n\n**Action Space** (REQUIRED RESPONSE FORMAT):\nturn_left   : Rotate 90° counter-clockwise\nturn_right  : Rotate 90° clockwise\nmove_forward: Advance if path clear\npickup      : Collect keys/objects\ndrop        : Drop carried object\ntoggle      : Open doors or interact (facing target required)\ndone        : ONLY when goal reached\n\n**Observation**:\nYou receive the entire grid as an observation, represented as a

# 上述交互分析 - from R1

<think>

嗯，我需要仔细分析用户提供的智能体在不同环境中的表现数据，尤其是后面两个环境（MiniGrid-DoorKey-6x6-v0和MiniGrid-LavaGapS5-v0）表现不佳的原因。首先，我应该先分别查看每个环境的数据和对应的对话记录，理解智能体的动作和结果。

首先看MiniGrid-Empty-5x5-v0，这个环境看起来比较简单，智能体的平均步数是8.75，平均奖励接近0.47，没有无效动作。这说明在空环境中，智能体能有效找到目标，可能路径规划正确。

接下来是MiniGrid-DoorKey-6x6-v0，平均步数15.0，奖励0，无效动作也是0。这说明智能体虽然完成了动作，但可能没有完成任务，比如没有正确使用钥匙开门，或者在步骤耗尽前未能到达目标。查看对话记录，发现智能体在第一步移动后尝试pickup，但似乎没有拿到钥匙。之后多次转向和移动，但似乎没有正确操作门。例如，在某个步骤中，智能体面对门时可能没有使用钥匙，或者门的状态未正确处理。可能需要检查是否有正确拾取钥匙并使用toggle开门，或者是否因为步骤限制导致无法完成任务。

第三个环境是MiniGrid-LavaGapS5-v0，平均步数10.0，奖励0，同样无效动作为0。这里任务需要避开岩浆到达目标。查看对话记录，智能体第一次移动到了(2,1)，但该位置旁边可能有岩浆，导致后续步骤可能掉入岩浆而失败。可能在第一步移动后，下一步动作没有正确避开岩浆，导致任务失败。

具体分析DoorKey环境的问题：智能体在拾取钥匙后，可能需要正确的方向去开门，但对话记录显示在尝试toggle时可能没有正确面对门，或者门的状态是锁定的而没使用钥匙。例如，在某个步骤中，智能体可能在错误的位置尝试toggle，或者没有携带钥匙导致无法开门。查看对话中的某个步骤，当智能体在(2,2)面朝右时，前面的单元格是(3,2)，这里的数据显示是门（类型4）。如果门是锁定的，智能体需要先使用钥匙才能toggle打开。但智能体在拾取钥匙后是否有携带？在对话记录中，状态显示“not carrying anything”，这可能说明pickup动作未成功，导致无法开门。例如，在第一次尝试pickup时，可能所在位置没有钥匙，或钥匙颜色不匹配。检查用户提供的对话记录中的观察数据，发现钥匙可能在(1,1)的位置，而智能体在(2,4)时尝试pickup，但该位置是否有钥匙？需要仔细核对每个步骤的观察数据。

对于LavaGap环境，智能体第一步移动到了(2,1)，而该位置的下方或旁边可能有岩浆，导致后续移动时掉入。例如，在移动后，可能需要转向或绕开岩浆区域，但智能体可能继续直行导致失败。观察数据中，在(2,1)的位置，可能前方或周围有岩浆，但智能体没有正确转向避开。

总结原因：在DoorKey环境中，智能体可能未能正确拾取钥匙或使用钥匙开门，导致无法通过门到达目标；而在LavaGap环境中，路径规划未避开岩浆，导致掉入陷阱。需要检查动作序列是否遵循了正确的开门流程和避让危险区域的策略。

</think>


  
嗯，下面是对两个表现不佳环境的分析：

---

### **1. MiniGrid-DoorKey-6x6-v0 表现分析**
**问题表现**  
- **平均奖励 0**：任务未完成（未成功开门抵达目标）
- **关键问题**：智能体未正确完成「拾钥→开门→通行」的流程。

**具体错误**：
1. **钥匙拾取失败**  
   - 初始位置 (1,4) 右移到 (2,4) 后尝试 `pickup`，但该位置无钥匙（钥匙在 (1,1)）。
   - 后续多次无效 `pickup` 浪费步骤（如 (2,2)、(2,1) 的拾取尝试均无钥匙）。

2. **方向与开门逻辑错误**  
   - 在 (2,2) 面朝右时，对 (3,2) 的锁门（类型 4，状态 2）执行 `toggle`，但未携带钥匙导致操作无效。
   - 未调整方向直对门或未正确使用钥匙。

3. **路径冗余**  
   - 多次无效转向（如反复 `turn_left`/`turn_right`）导致步数耗尽。

---

### **2. MiniGrid-LavaGapS5-v0 表现分析**
**问题表现**  
- **平均奖励 0**：未避开岩浆导致任务失败。
- **关键问题**：路径规划未识别危险区域。

**具体错误**：
1. **直行落入岩浆**  
   - 初始位置 (1,1) 右移到 (2,1)，但 (2,1) 下方 (2,2) 是岩浆（类型 9）。
   - 后续若继续直行或转向错误会触发死亡。

2. **缺乏避障策略**  
   - 未通过 `turn_left`/`turn_right` 绕路（例如向上避开 (2,1)→(2,2) 的岩浆）。

---

### **改进建议**
1. **DoorKey 环境**  
   - 优先导航至钥匙位置（需向左/上移动而非右移）。
   - 携带钥匙后面朝锁门执行 `toggle`。

2. **LavaGap 环境**  
   - 检测前方岩浆后立即转向（如从 (1,1) 上移避开 (2,1)→(2,2) 的岩浆）。
   - 优先选择安全路径（例如向上绕行）。

---

**根本原因**：智能体对钥匙定位、门交互逻辑和危险区域规避的规则理解不足，需加强状态解析与路径规划能力。

## 单步测试

In [7]:
from main import *

In [8]:
agent = MiniGridLLMAgent("MiniGrid-Empty-5x5-v0", max_steps=10)
obs_text, obs, info = agent.reset(seed=42)
print(obs_text)

Mission: get to the green goal square
Grid size: 5x5
Agent at (1, 1), facing right (→)
Status: not carrying anything | 10 steps remaining
Observation:
[[[ 0,  0,  2],
  [ 0,  1,  2],
  [ 0,  2,  2],
  [ 0,  3,  2],
  [ 0,  4,  2]],

 [[ 1,  0,  2],
  [ 1,  1, 10],
  [ 1,  2, -1],
  [ 1,  3, -1],
  [ 1,  4,  2]],

 [[ 2,  0,  2],
  [ 2,  1, -1],
  [ 2,  2, -1],
  [ 2,  3, -1],
  [ 2,  4,  2]],

 [[ 3,  0,  2],
  [ 3,  1, -1],
  [ 3,  2, -1],
  [ 3,  3,  8],
  [ 3,  4,  2]],

 [[ 4,  0,  2],
  [ 4,  1,  2],
  [ 4,  2,  2],
  [ 4,  3,  2],
  [ 4,  4,  2]]]



In [9]:
obs_text, reward, done, obs, info = agent.step('move_forward')
print(obs_text)

Mission: get to the green goal square
Grid size: 5x5
Agent at (2, 1), facing right (→)
Status: not carrying anything | 9 steps remaining
Observation:
[[[ 0,  0,  2],
  [ 0,  1,  2],
  [ 0,  2,  2],
  [ 0,  3,  2],
  [ 0,  4,  2]],

 [[ 1,  0,  2],
  [ 1,  1, -1],
  [ 1,  2, -1],
  [ 1,  3, -1],
  [ 1,  4,  2]],

 [[ 2,  0,  2],
  [ 2,  1, 10],
  [ 2,  2, -1],
  [ 2,  3, -1],
  [ 2,  4,  2]],

 [[ 3,  0,  2],
  [ 3,  1, -1],
  [ 3,  2, -1],
  [ 3,  3,  8],
  [ 3,  4,  2]],

 [[ 4,  0,  2],
  [ 4,  1,  2],
  [ 4,  2,  2],
  [ 4,  3,  2],
  [ 4,  4,  2]]]



In [10]:
obs_text, reward, done, obs, info = agent.step('move_forward')
print(obs_text)

Mission: get to the green goal square
Grid size: 5x5
Agent at (3, 1), facing right (→)
Status: not carrying anything | 8 steps remaining
Observation:
[[[ 0,  0,  2],
  [ 0,  1,  2],
  [ 0,  2,  2],
  [ 0,  3,  2],
  [ 0,  4,  2]],

 [[ 1,  0,  2],
  [ 1,  1, -1],
  [ 1,  2, -1],
  [ 1,  3, -1],
  [ 1,  4,  2]],

 [[ 2,  0,  2],
  [ 2,  1, -1],
  [ 2,  2, -1],
  [ 2,  3, -1],
  [ 2,  4,  2]],

 [[ 3,  0,  2],
  [ 3,  1, 10],
  [ 3,  2, -1],
  [ 3,  3,  8],
  [ 3,  4,  2]],

 [[ 4,  0,  2],
  [ 4,  1,  2],
  [ 4,  2,  2],
  [ 4,  3,  2],
  [ 4,  4,  2]]]

